## Combined code

In [3]:
import re
import pandas as pd
from os import listdir

# return all filename with .csv
def find_csv_filenames(path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

old_filenames = find_csv_filenames("./csv")
filenames = old_filenames[19:]+old_filenames[:19] # sort them into 86,87,88...17,18 order
# print(filenames)

# function process every file
def process(filename):
    year = filename[3:5] # get year from filename e.g.18
    print(year)
    fields = ["fipstate", "naics", "est"] # only select these three columns
    df = pd.read_csv("./csv/"+filename, skipinitialspace=True, usecols=fields) # read csv file
    df['year'] = year # add a column called 'year'   
    regex = "^\d{3,}" # define regular express to filter out digits less than 3
    filtered = df[df.naics.str.contains(regex)] # filter
    filtered["naics"] = filtered.naics.str[:3] # only select first three digits
    grouped = filtered.groupby(['year', 'fipstate', 'naics']).sum().reset_index() # group by state and sum up est for each state
    state_code = pd.read_excel('code.xlsx') # get fips code 
    merged = pd.merge(left=grouped, right=state_code, left_on='fipstate', right_on='FIPS') # join two table based on fips 
    merged.drop(['fipstate', 'FIPS'],axis=1, inplace=True) # delete these columns since not interested
    merged = merged[['year', 'Name', 'Postal Code', 'naics', 'est']] # rearrange column order
    return merged

result = pd.DataFrame([], columns=["year","Name","Postal Code","naics","est"]) # init dataframe
for filename in filenames: # iterate every file
    df = process(filename)
    result = result.append(df, ignore_index = True) # append each year result to final result
result.to_excel(r'output.xlsx', index = False) # export as excel file

86


c:\users\xiey0\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


87
88
89
90
91
92
93
94
95
96
97
98
99
00
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18


## Separate code

In [9]:
filename = 'cbp18st.csv'
year = filename[3:5] # 18
fields = ["fipstate", "naics", "est"]
df = pd.read_csv("./csv/"+filename, skipinitialspace=True, usecols=fields)
df['year'] = year
df

,fipstate,naics,est,year
0,1,------,100267,18
1,1,------,27971,18
2,1,------,39352,18
3,1,------,10941,18
4,1,------,12310,18
...,...,...,...,...
348072,56,99----,9,18
348073,56,99----,7,18
348074,56,99----,26,18
348075,56,99----,25,18


In [11]:
regex = "^\d{3,}"
filtered = df[df.naics.str.contains(regex)]
filtered["naics"] = filtered.naics.str[:3]
filtered

c:\users\xiey0\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,fipstate,naics,est,year
15,1,113,573,18
16,1,113,99,18
17,1,113,235,18
18,1,113,140,18
19,1,113,98,18
...,...,...,...,...
348066,56,813,9,18
348067,56,813,18,18
348068,56,813,16,18
348069,56,813,18,18


In [12]:
grouped = filtered.groupby(['year', 'fipstate', 'naics']).sum().reset_index()
grouped

,year,fipstate,naics,est
0,18,1,113,4574
1,18,1,114,175
2,18,1,115,1592
3,18,1,211,158
4,18,1,212,893
...,...,...,...,...
4251,18,56,721,4148
4252,18,56,722,10269
4253,18,56,811,5135
4254,18,56,812,2777


In [13]:
state_code = pd.read_excel('code.xlsx')
state_code

,Name,Postal Code,FIPS
0,Alabama,AL,1
1,Alaska,AK,2
2,Arizona,AZ,4
3,Arkansas,AR,5
4,California,CA,6
5,Colorado,CO,8
6,Connecticut,CT,9
7,Delaware,DE,10
8,Florida,FL,12
9,Georgia,GA,13


In [14]:
merged = pd.merge(left=grouped, right=state_code, left_on='fipstate', right_on='FIPS')
merged.drop(['fipstate', 'FIPS'],axis=1, inplace=True)
merged = merged[['year', 'Name', 'Postal Code', 'naics', 'est']]
merged

,year,Name,Postal Code,naics,est
0,18,Alabama,AL,113,4574
1,18,Alabama,AL,114,175
2,18,Alabama,AL,115,1592
3,18,Alabama,AL,211,158
4,18,Alabama,AL,212,893
...,...,...,...,...,...
4184,18,Wyoming,WY,721,4148
4185,18,Wyoming,WY,722,10269
4186,18,Wyoming,WY,811,5135
4187,18,Wyoming,WY,812,2777


In [ ]:
merged.to_excel(r'output.xlsx', index = False)